In [1]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.


In [2]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from zipfile import ZipFile
dataset = 'sentiment140.zip'
with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print('data extracted')

data extracted


In [20]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
colum_name = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('tweets.csv',names=colum_name,encoding='ISO-8859-1')

In [15]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [17]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [6]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [19]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [7]:
ps = PorterStemmer()

In [8]:
def stemming(content):
    s_content = re.sub('[^a-zA-Z]',' ',content)
    s_content = s_content.lower()
    s_content = s_content.split()
    s_content = [ps.stem(word) for word in s_content if not word in stopwords.words('english')]
    s_content = ' '.join(s_content)

    return s_content

In [10]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [13]:
print(twitter_data['stemmed_content'].head())

0    switchfoot http twitpic com zl awww bummer sho...
1    upset updat facebook text might cri result sch...
2    kenichan dive mani time ball manag save rest g...
3                      whole bodi feel itchi like fire
4                        nationwideclass behav mad see
Name: stemmed_content, dtype: object


In [14]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [27]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=3)

In [28]:
vec = TfidfVectorizer()

x_train = vec.fit_transform(x_train)
x_test = vec.transform(x_test)

In [29]:
model = LogisticRegression(max_iter=1000)

In [30]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [32]:
train_pred = model.predict(x_train)
acc_score = accuracy_score(y_train,train_pred)
print(acc_score)

0.8025671875


In [33]:
test_pred = model.predict(x_test)
test_acc_score = accuracy_score(y_test,test_pred)
print(test_acc_score)

0.777428125


In [34]:
import pickle


In [35]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [36]:
loaded_model = pickle.load(open('trained_model.sav','rb'))

In [42]:
x_new = x_test[20000]
print(y_test[20000])

prediction = loaded_model.predict(x_new)
print(prediction)

1
[1]
